In [38]:
from collections import Counter
import csv
import re

In [39]:
with open("/home/niraj/Downloads/training.txt",'r')as file:
    reviews = list(csv.reader(file,delimiter ='\t'))
    
reviews

[['1', 'The Da Vinci Code book is just awesome.'],
 ['1',
  "this was the first clive cussler i've ever read, but even books like Relic, and Da Vinci code were more plausible than this."],
 ['1', 'i liked the Da Vinci Code a lot.'],
 ['1', 'i liked the Da Vinci Code a lot.'],
 ['1',
  "I liked the Da Vinci Code but it ultimatly didn't seem to hold it's own."],
 ['1',
  "that's not even an exaggeration ) and at midnight we went to Wal-Mart to buy the Da Vinci Code, which is amazing of course."],
 ['1',
  'I loved the Da Vinci Code, but now I want something better and different!..'],
 ['1', 'i thought da vinci code was great, same with kite runner.'],
 ['1', 'The Da Vinci Code is actually a good movie...'],
 ['1', 'I thought the Da Vinci Code was a pretty good book.'],
 ['1', 'The Da Vinci Code is one of the most beautiful movies ive ever seen.'],
 ['1', 'The Da Vinci Code is an * amazing * book, do not get me wrong.'],
 ['1', 'then I turn on the light and the radio and enjoy my Da Vinci

In [40]:
print(len(reviews))

7086


In [41]:
def get_text(reviews, score):
  # Join together the text in the reviews for a particular tone.
  # We lowercase to avoid "Not" and "not" being seen as different words, for example.
  return " ".join([r[1].lower() for r in reviews if r[0] == str(score)])

In [42]:
def count_text(text):
  # Split text into words based on whitespace.  Simple but effective.
  words = re.split("\s+", text)
  # Count up the occurence of each word.
  return Counter(words)

In [43]:
negative_text = get_text(reviews, 0)
positive_text = get_text(reviews, 1)

# Generate word counts for negative tone.
negative_counts = count_text(negative_text)
print(negative_counts)

# Generate word counts for positive tone.
positive_counts = count_text(positive_text)
print(positive_counts)

print("Negative text sample: {0}".format(negative_text[:100]))
print("Positive text sample: {0}".format(positive_text[:100]))

Counter({'i': 1417, 'the': 1385, 'and': 1194, 'harry': 1003, 'vinci': 1001, 'da': 999, 'brokeback': 997, 'code': 946, 'mountain': 716, 'hate': 571, 'is': 525, 'potter': 524, 'a': 453, 'stupid': 358, 'my': 350, 'was': 335, 'it': 294, 'sucked': 289, 'sucks': 285, 'to': 277, 'movie': 276, 'of': 266, 'just': 265, 'because': 264, 'much': 262, 'potter,': 255, 'that': 243, 'really': 187, 'sucks.': 187, 'with': 184, 'movie.': 180, 'his': 175, 'not': 172, 'out': 171, 'am': 169, 'fucking': 169, 'terrible': 168, 'movies': 167, 'only': 167, 'into': 165, 'she': 163, 'oh,': 162, 'down,': 160, 'up,': 160, 'ok': 160, 'left,': 160, 'right,': 160, 'but': 150, 'so': 120, 'which': 119, 'you': 117, 'this': 113, 'potter..': 113, 'about': 112, 'think': 111, 'as': 110, 'how': 105, 'we': 104, 'potter.': 101, 'have': 100, 'one': 99, 'are': 99, 'horrible': 95, 'heard': 94, 'sucks...': 94, '/': 93, 'me': 93, 'at': 93, 'from': 93, 'then': 92, 'mission': 92, 'mountain.': 92, 'depressing': 92, 'mountain,': 91, 'by':

In [44]:
def get_y_count(score):
  # Compute the count of each classification occuring in the data.
  return len([r for r in reviews if r[0] == str(score)])

# We need these counts to use for smoothing when computing the prediction.
positive_review_count = get_y_count(1)
negative_review_count = get_y_count(0)

print(positive_review_count)
print(negative_review_count)

3995
3091


In [45]:
# These are the class probabilities (we saw them in the formula as P(y)).
prob_positive = positive_review_count / len(reviews)
prob_negative = negative_review_count / len(reviews)

print(prob_positive)
print(prob_negative)

0.5637877504939317
0.4362122495060683


In [46]:
def make_class_prediction(text, counts, class_prob, class_count):
  prediction = 1
  text_counts = Counter(re.split("\s+", text))
  for word in text_counts:
      # For every word in the text, we get the number of times that word occured in the reviews for a given class, add 1 to smooth the value, and divide by the total number of words in the class (plus the class_count to also smooth the denominator).
      # Smoothing ensures that we don't multiply the prediction by 0 if the word didn't exist in the training data.
      # We also smooth the denominator counts to keep things even.
      prediction *=  text_counts.get(word) * ((counts.get(word, 0) + 1) / (sum(counts.values()) + class_count))
  # Now we multiply by the probability of the class existing in the documents.
  return prediction * class_prob

In [47]:
# As you can see, we can now generate probabilities for which class a given review is part of.
# The probabilities themselves aren't very useful -- we make our classification decision based on which value is greater.
print("Review: {0}".format(reviews[0][1]))
neg = make_class_prediction(reviews[0][1], negative_counts, prob_negative, negative_review_count)
pos = make_class_prediction(reviews[0][1], positive_counts, prob_positive, positive_review_count)
print("Negative prediction: {0}".format(neg))
print("Positive prediction: {0}".format(pos))

if neg > pos:
    print("Negative")
else:
    print("Positive")

Review: The Da Vinci Code book is just awesome.
Negative prediction: 8.27689157897865e-32
Positive prediction: 1.6992915918347523e-30
Positive


In [63]:
def make_decision(text, make_class_prediction):
    # Compute the negative and positive probabilities.
    negative_prediction = make_class_prediction(text, negative_counts, prob_negative, negative_review_count)
    positive_prediction = make_class_prediction(text, positive_counts, prob_positive, positive_review_count)

    # We assign a classification based on which probability is greater.
    if negative_prediction > positive_prediction:
      return 0
    return 1

In [64]:
with open("/home/niraj/Downloads/testdata.txt", 'r') as file:
    test = list(csv.reader(file,delimiter='\n'))
    
print(len(test))   

33052


In [65]:
predictions = [make_decision(r[0], make_class_prediction) for r in test]

In [58]:
predictions

[1, 1, -1, -1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, -1, 1, -1, 1, 1, 1]

In [70]:
with open("/home/niraj/Downloads/testing.txt", 'r') as file:
    test = list(csv.reader(file,delimiter='\t'))
    
print(len(test))    


20


In [71]:
predictions = [make_decision(r[0], make_class_prediction) for r in test]

In [72]:
predictions

[1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1]

In [73]:
actual = [int(r[0]) for r in test]

from sklearn import metrics

# Generate the roc curve using scikits-learn.
fpr, tpr, thresholds = metrics.roc_curve(actual, predictions, pos_label=1)


# Measure the area under the curve.  The closer to 1, the "better" the predictions.
print("AUC of the predictions: {0}".format(metrics.auc(fpr, tpr)))


AUC of the predictions: 1.0
